# Zadanie Domowe

2. Klasyfikacja grzybów (https://archive.ics.uci.edu/ml/datasets/Mushroom)
Przy użyciu danych mushrooms.csv zbuduj następujące modele: AdaBoost, XGBoost, LightGBM oraz CatBoost. Porównaj ich wyniki. Spróbuj ulepszyć wyniki każdego z wariantów poprzed dostrojenie parametrów - skorzystaj z dokumentacji wymienionych metod. 

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [119]:
df = pd.read_csv("mushrooms.csv")

In [83]:
df.shape

(8124, 23)

In [84]:
df.isna().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [86]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [87]:
df.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [120]:
features = df.drop("class", axis = 1)
features_dummies = pd.get_dummies(df.drop("class", axis = 1))
target = df["class"]

In [121]:
target = target.replace(('e', 'p'), (1, 0))

In [122]:
target

0       0
1       1
2       1
3       0
4       1
       ..
8119    1
8120    1
8121    1
8122    0
8123    1
Name: class, Length: 8124, dtype: int64

In [136]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_dummies, target, test_size = 0.8, random_state = 1, stratify = target)

In [131]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [132]:
def evaluate(prediction, y_test): 
    result = classification_report(y_test, prediction, output_dict = True)
    f1 = result['1']['f1-score']
    accuracy = result['accuracy']
    performance_data= {'f1-score':round(f1, 4),
                      'accuracy':round(accuracy, 4)}
    return performance_data

## Logistic Regression

In [162]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 1)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_pr = evaluate(lr_pred, y_test)
lr_pr

{'f1-score': 0.9991, 'accuracy': 0.9991}

In [163]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

In [164]:
# l2 regularization + CV
from sklearn.model_selection import GridSearchCV
lr2 = LogisticRegression(random_state = 1)
lr2_cv = GridSearchCV(lr2, {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}, cv = kf)
lr2_cv.fit(X_train, y_train)
print(lr2_cv.best_params_)
lr2_cv_pred = lr2_cv.predict(X_test)
lr2_cv_pr = evaluate(lr2_cv_pred, y_test)
lr2_cv_pr

{'C': 100}


{'f1-score': 1.0, 'accuracy': 1.0}

In [165]:
# l1 regularization + CV
lr3 = LogisticRegression(solver = 'liblinear', penalty = 'l1', random_state = 1)
lr3_cv = GridSearchCV(lr3, {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}, cv = kf)
lr3_cv.fit(X_train, y_train)
print(lr3_cv.best_params_)
lr3_cv_pred = lr3_cv.predict(X_test)
lr3_cv_pr = evaluate(lr3_cv_pred, y_test)
lr3_cv_pr

{'C': 100}


{'f1-score': 1.0, 'accuracy': 1.0}

## Decision Tree

In [141]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state = 1)
dtree.fit(X_train, y_train)
dtree_pred = dtree.predict(X_test)
dtree_pr = evaluate(dtree_pred, y_test)
dtree_pr

{'f1-score': 0.9996, 'accuracy': 0.9995}

In [142]:
# Decision Tree + RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
param_rand = {'max_features' : ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'max_depth' : np.arange(10, 100, 10)}
dt_random = RandomizedSearchCV(estimator = DecisionTreeClassifier(random_state = 1), param_distributions = param_rand, n_iter = 100,
                               cv = kf, random_state = 1, n_jobs = -1)
dt_random.fit(X_train, y_train)
dt_random_pred = dt_random.predict(X_test)
dt_random_pr = evaluate(dt_random_pred, y_test)
dt_random_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [143]:
dt_cv.best_params_

{'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [144]:
# Decision Tree + GridSearchCV
param_grid = {'max_features' : ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'max_depth' : np.arange(10, 100, 10)}
dt_cv = GridSearchCV(estimator = DecisionTreeClassifier(random_state=1), param_grid = param_grid, cv = kf, n_jobs = -1)
dt_cv.fit(X_train, y_train)
dt_cv_pred = dt_cv.predict(X_test)
dt_cv_pr = evaluate(dt_cv_pred, y_test)
dt_cv_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [145]:
dt_random.best_params_

{'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 60}

In [146]:
# Decision Tree + BC
from sklearn.ensemble import BaggingClassifier
dtree_bc = BaggingClassifier(DecisionTreeClassifier(random_state = 1), random_state = 1)
dtree_bc.fit(X_train, y_train)
dtree_bc_pred = dtree_bc.predict(X_test)
dtree_bc_pr = evaluate(dtree_bc_pred, y_test)
dtree_bc_pr

{'f1-score': 0.9996, 'accuracy': 0.9995}

## Random Forest

In [147]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, random_state = 1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_pr = evaluate(rf_pred, y_test)
rf_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [148]:
# Random Forest + RandomizedSearchCV
param_rand = {'n_estimators': np.arange(100,800,100),
              'max_features' : ['auto', 'sqrt'],
              'min_samples_leaf': [1,2,4],
              'min_samples_split': [2, 5, 10],
              'max_depth' : np.arange(10,100,10)}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(random_state = 1), param_distributions = param_rand, n_iter = 100,
                               cv = kf, random_state = 1, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random_pred = rf_random.predict(X_test)
rf_random_pr = evaluate(rf_random_pred, y_test)
rf_random_pr

{'f1-score': 1.0, 'accuracy': 1.0}

In [149]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 40}

In [150]:
# Random Forest + GridSearchCV - zajmuje duuużo czasu
# param_grid = {'n_estimators': np.arange(100,800,100),
#              'max_features' : ['auto', 'sqrt'],
#              'min_samples_leaf': [1,2,4],
#              'min_samples_split': [2, 5, 10],
#              'max_depth' : np.arange(10,100,10)}
# rf_cv = GridSearchCV(estimator = RandomForestClassifier(random_state = 1), param_grid = param_grid,
#                               cv = kf, n_jobs = -1)
# rf_cv.fit(X_train, y_train)
# rf_cv_pred = rf_cv.predict(X_test)
# rf_cv_pr = evaluate(rf_cv_pred, y_test)
# rf_cv_pr

In [151]:
# rf_cv.best_params_

## SVM

In [152]:
from sklearn.svm import SVC
svc = SVC(random_state = 1)
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)
svc_pr = evaluate(svc_pred, y_test)
svc_pr

{'f1-score': 0.9991, 'accuracy': 0.9991}

In [153]:
# 'rbf' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc2 = SVC(random_state = 1)
svc2_cv = GridSearchCV(svc2, param_grid)
svc2_cv.fit(X_train, y_train)
print(svc2_cv.best_params_)
svc2_cv_pred = svc2_cv.predict(X_test)
svc2_cv_pr = evaluate(svc2_cv_pred, y_test)
svc2_cv_pr

{'C': 1, 'gamma': 0.1}


{'f1-score': 1.0, 'accuracy': 1.0}

In [154]:
# 'linear' + CV
svc3 = SVC(kernel = 'linear', random_state = 1)
svc3_cv = GridSearchCV(svc3, {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]})
svc3_cv.fit(X_train, y_train)
print(svc3_cv.best_params_)
svc3_cv_pred = svc3_cv.predict(X_test)
svc3_cv_pr = evaluate(svc3_cv_pred, y_test)
svc3_cv_pr

{'C': 1}


{'f1-score': 1.0, 'accuracy': 1.0}

In [155]:
# 'poly' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc4 = SVC(kernel = 'poly', random_state = 1)
svc4_cv = GridSearchCV(svc4, param_grid)
svc4_cv.fit(X_train, y_train)
print(svc4_cv.best_params_)
svc4_cv_pred = svc4_cv.predict(X_test)
svc4_cv_pr = evaluate(svc4_cv_pred, y_test)
svc4_cv_pr

{'C': 0.001, 'gamma': 1}


{'f1-score': 1.0, 'accuracy': 1.0}

In [156]:
# 'sigmoid' + CV
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
svc5 = SVC(kernel = 'sigmoid', random_state = 1)
svc5_cv = GridSearchCV(svc5, param_grid)
svc5_cv.fit(X_train, y_train)
print(svc5_cv.best_params_)
svc5_cv_pred = svc5_cv.predict(X_test)
svc5_cv_pr = evaluate(svc5_cv_pred, y_test)
svc5_cv_pr

{'C': 100, 'gamma': 0.01}


{'f1-score': 1.0, 'accuracy': 1.0}

In [157]:
# all
param_grid = {'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1], 'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel':['rbf','linear','poly','sigmoid']}
svc_all_params = SVC(random_state = 1)
svc_all_params_cv = GridSearchCV(svc_all_params, param_grid)
clf = svc_all_params_cv.fit(X_train, y_train)
print(svc_all_params_cv.best_params_)
svc_all_params_cv_pred = svc_all_params_cv.predict(X_test)
svc_all_params_cv_pr = evaluate(svc_all_params_cv_pred, y_test)
svc_all_params_cv_pr

{'C': 0.001, 'gamma': 1, 'kernel': 'poly'}


{'f1-score': 1.0, 'accuracy': 1.0}

## KNN

In [158]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
knn_pr = evaluate(knn_pred, y_test)
knn_pr

{'f1-score': 0.9997, 'accuracy': 0.9997}

In [159]:
accuracies = {}
for neighbor in np.arange(1, 10):
    knn = KNeighborsClassifier(n_neighbors = neighbor)
    knn.fit(X_train, y_train)
    accuracies[neighbor] = knn.score(X_test, y_test)
accuracies

{1: 1.0,
 2: 1.0,
 3: 0.9996923076923077,
 4: 1.0,
 5: 0.9986153846153846,
 6: 0.9986153846153846,
 7: 0.9981538461538462,
 8: 0.9983076923076923,
 9: 0.9981538461538462}

In [160]:
# KNN + CV
knn2 = KNeighborsClassifier()
knn_cv = GridSearchCV(knn2, {'n_neighbors': np.arange(1,10)}, cv = kf)
knn_cv.fit(X_train, y_train)
print(knn_cv.best_params_)
knn_cv_pred = knn_cv.predict(X_test)
knn_cv_pr = evaluate(knn_cv_pred, y_test)
knn_cv_pr

{'n_neighbors': 2}


{'f1-score': 1.0, 'accuracy': 1.0}

In [167]:
prediction_data = {'Model':['Logistic Regression', 'Logistic Regression - l2 regularization + CV', 'Logistic Regression - l1 regularization + CV', 'Decision Tree', 'Decision Tree + RandomizedSearchCV', 'Decision Tree + GridSearchCV', 'Decision Tree + BC', 'Random Forest', 'Random Forest + RandomizedSearchCV', 'SVM rbf', 'SVM rbf + CV', 'SVM linear + CV', 'SVM poly + CV', 'SVM sigmoid + CV', 'SVM all params CV','KNN', 'KNN + CV'],
                   'F1-score':[lr_pr['f1-score'], lr2_cv_pr['f1-score'], lr3_cv_pr['f1-score'], dtree_pr['f1-score'], dt_random_pr['f1-score'], dt_cv_pr['f1-score'], dtree_bc_pr['f1-score'], rf_pr['f1-score'], rf_random_pr['f1-score'], svc_pr['f1-score'], svc2_cv_pr['f1-score'], svc3_cv_pr['f1-score'], svc4_cv_pr['f1-score'], svc5_cv_pr['f1-score'], svc_all_params_cv_pr['f1-score'], knn_pr['f1-score'], knn_cv_pr['f1-score']],
                   'Accuracy':[lr_pr['accuracy'], lr2_cv_pr['accuracy'], lr3_cv_pr['accuracy'], dtree_pr['accuracy'], dt_random_pr['accuracy'], dt_cv_pr['accuracy'], dtree_bc_pr['accuracy'], rf_pr['accuracy'], rf_random_pr['accuracy'], svc_pr['accuracy'], svc2_cv_pr['accuracy'], svc3_cv_pr['accuracy'], svc4_cv_pr['accuracy'], svc5_cv_pr['accuracy'], svc_all_params_cv_pr['accuracy'], knn_pr['accuracy'], knn_cv_pr['accuracy']]
                    }
 
prediction_table = pd.DataFrame(prediction_data)
prediction_table

,Model,F1-score,Accuracy
0,Logistic Regression,0.9991,0.9991
1,Logistic Regression - l2 regularization + CV,1.0000,1.0000
2,Logistic Regression - l1 regularization + CV,1.0000,1.0000
3,Decision Tree,0.9996,0.9995
4,Decision Tree + RandomizedSearchCV,1.0000,1.0000
5,Decision Tree + GridSearchCV,1.0000,1.0000
6,Decision Tree + BC,0.9996,0.9995
7,Random Forest,1.0000,1.0000
8,Random Forest + RandomizedSearchCV,1.0000,1.0000
9,SVM rbf,0.9991,0.9991


## AdaBoost

In [168]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(random_state = 1)
ada.fit(X_train, y_train)
ada.score(X_test,y_test)

1.0

In [169]:
accuracies = {}
for n_estimators in np.arange(1, 10):
    ada = AdaBoostClassifier(random_state = 1, n_estimators = n_estimators)
    ada.fit(X_train, y_train)
    accuracies[n_estimators] = ada.score(X_test, y_test)
accuracies

{1: 0.8856923076923077,
 2: 0.8856923076923077,
 3: 0.939076923076923,
 4: 0.9578461538461538,
 5: 0.9670769230769231,
 6: 0.9515384615384616,
 7: 0.9781538461538462,
 8: 0.9823076923076923,
 9: 0.9949230769230769}

## Gradient Boosting

In [170]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(learning_rate = 0.01 , random_state = 1)
gbc.fit(X_train, y_train)
gbc.score(X_test,y_test)

0.9969230769230769

In [180]:
accuracies = {}
for learning_rate in [0.01,0.1,0.3,0.5]:
    gbc = GradientBoostingClassifier(random_state = 1, learning_rate = learning_rate)
    gbc.fit(X_train, y_train)
    accuracies[learning_rate] = gbc.score(X_test, y_test)
accuracies

{0.01: 0.9969230769230769, 0.1: 0.9990769230769231, 0.3: 1.0, 0.5: 1.0}

## XGBoost

In [549]:
!pip install xgboost

In [181]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(random_state = 1,learning_rate = 0.01)
xgbc.fit(X_train, y_train)
xgbc.score(X_test, y_test)

0.9969230769230769

In [182]:
accuracies = {}
for learning_rate in [0.01,0.1,0.3,0.5]:
    xgbc = xgb.XGBClassifier(random_state = 1, learning_rate = learning_rate)
    xgbc.fit(X_train, y_train)
    accuracies[learning_rate] = xgbc.score(X_test, y_test)
accuracies

{0.01: 0.9969230769230769,
 0.1: 0.9990769230769231,
 0.3: 0.9990769230769231,
 0.5: 0.9990769230769231}

In [176]:
# xgboost cross-validation
dmatrix = xgb.DMatrix(data = features_dummies, label = target)
params = {"objective":"reg:logistic", "max_depth":3}
cv_results = xgb.cv(dtrain=dmatrix, params=params, 
                  nfold=3, num_boost_round=5, 
                  metrics="auc", as_pandas=True, seed=123)
print(cv_results)

   train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0        0.986594       0.000814       0.986061      0.002084
1        0.997903       0.000227       0.997836      0.000314
2        0.998389       0.000528       0.998272      0.000173
3        0.998764       0.000233       0.998736      0.000433
4        0.998774       0.000225       0.998766      0.000452


## LightGBM

In [1]:
!pip install lightgbm

In [2]:
import lightgbm as lgb

In [34]:
train_data = lgb.Dataset(X_train, label = y_train)
#define parameters
params = {'learning_rate':0.001}
model = lgb.train(params, train_data, 100) 
y_pred = model.predict(X_test)
for i in range(0, 185):
    if y_pred[i] >= 0.5:
        y_pred[i] = 1
    else: 
        y_pred[i] = 0
y_pred = pd.DataFrame(y_pred)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_pred, y_test)**0.5)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018840 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 168
[LightGBM] [Info] Number of data points in the train set: 812, number of used features: 84
[LightGBM] [Info] Start training from score 0.518473
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [37]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9967177242888403

## CatBoost

In [38]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 6.9 MB/s eta 0:00:0000:0100:01
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 3.4 MB/s eta 0:00:0000:0100:01
  Using cached tenacity-8.1.0-py3-none-any.whl (23 kB)


In [40]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
categorical_features_indices = np.where(df.dtypes != np.float)[0]
model.fit(X_train, y_train, cat_features=([ 0,  1, 2, 3, 4, 10]), eval_set=(X_test, y_test))
model.score(X_test, y_test)

/var/folders/gf/t0gbh7m514j9kjx2g299ch6h0000gn/T/ipykernel_89169/3782514624.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(df.dtypes != np.float)[0]


Learning rate set to 0.0301
0:	learn: 0.6222438	test: 0.6230600	best: 0.6230600 (0)	total: 59.5ms	remaining: 59.5s
1:	learn: 0.5646468	test: 0.5663041	best: 0.5663041 (1)	total: 62.5ms	remaining: 31.2s
2:	learn: 0.5130927	test: 0.5150308	best: 0.5150308 (2)	total: 65.4ms	remaining: 21.7s
3:	learn: 0.4680788	test: 0.4698742	best: 0.4698742 (3)	total: 68ms	remaining: 16.9s
4:	learn: 0.4251793	test: 0.4274372	best: 0.4274372 (4)	total: 70.1ms	remaining: 14s
5:	learn: 0.3834866	test: 0.3861993	best: 0.3861993 (5)	total: 72.2ms	remaining: 12s
6:	learn: 0.3535488	test: 0.3570047	best: 0.3570047 (6)	total: 74.2ms	remaining: 10.5s
7:	learn: 0.3240520	test: 0.3276598	best: 0.3276598 (7)	total: 77ms	remaining: 9.55s
8:	learn: 0.2937162	test: 0.2973771	best: 0.2973771 (8)	total: 79.7ms	remaining: 8.77s
9:	learn: 0.2680322	test: 0.2723716	best: 0.2723716 (9)	total: 82.7ms	remaining: 8.18s
10:	learn: 0.2453597	test: 0.2496779	best: 0.2496779 (10)	total: 85ms	remaining: 7.64s
11:	learn: 0.2286156	te

112:	learn: 0.0116882	test: 0.0163305	best: 0.0163305 (112)	total: 357ms	remaining: 2.8s
113:	learn: 0.0116853	test: 0.0163284	best: 0.0163284 (113)	total: 361ms	remaining: 2.81s
114:	learn: 0.0116723	test: 0.0163023	best: 0.0163023 (114)	total: 381ms	remaining: 2.93s
115:	learn: 0.0114323	test: 0.0160720	best: 0.0160720 (115)	total: 398ms	remaining: 3.03s
116:	learn: 0.0113139	test: 0.0159008	best: 0.0159008 (116)	total: 400ms	remaining: 3.02s
117:	learn: 0.0112476	test: 0.0158267	best: 0.0158267 (117)	total: 402ms	remaining: 3s
118:	learn: 0.0112151	test: 0.0157870	best: 0.0157870 (118)	total: 416ms	remaining: 3.08s
119:	learn: 0.0111839	test: 0.0157514	best: 0.0157514 (119)	total: 428ms	remaining: 3.14s
120:	learn: 0.0110633	test: 0.0156271	best: 0.0156271 (120)	total: 441ms	remaining: 3.2s
121:	learn: 0.0110025	test: 0.0155375	best: 0.0155375 (121)	total: 445ms	remaining: 3.2s
122:	learn: 0.0109116	test: 0.0154316	best: 0.0154316 (122)	total: 447ms	remaining: 3.19s
123:	learn: 0.01

210:	learn: 0.0054157	test: 0.0099402	best: 0.0099402 (210)	total: 979ms	remaining: 3.66s
211:	learn: 0.0053941	test: 0.0099292	best: 0.0099292 (211)	total: 983ms	remaining: 3.65s
212:	learn: 0.0053419	test: 0.0098588	best: 0.0098588 (212)	total: 1.03s	remaining: 3.79s
213:	learn: 0.0052423	test: 0.0098024	best: 0.0098024 (213)	total: 1.03s	remaining: 3.78s
214:	learn: 0.0052163	test: 0.0097505	best: 0.0097505 (214)	total: 1.04s	remaining: 3.78s
215:	learn: 0.0051406	test: 0.0096540	best: 0.0096540 (215)	total: 1.04s	remaining: 3.78s
216:	learn: 0.0051222	test: 0.0096272	best: 0.0096272 (216)	total: 1.04s	remaining: 3.77s
217:	learn: 0.0050364	test: 0.0095583	best: 0.0095583 (217)	total: 1.05s	remaining: 3.77s
218:	learn: 0.0050245	test: 0.0095210	best: 0.0095210 (218)	total: 1.05s	remaining: 3.76s
219:	learn: 0.0049376	test: 0.0094746	best: 0.0094746 (219)	total: 1.06s	remaining: 3.77s
220:	learn: 0.0049080	test: 0.0094289	best: 0.0094289 (220)	total: 1.07s	remaining: 3.76s
221:	learn

311:	learn: 0.0026960	test: 0.0074352	best: 0.0074352 (311)	total: 1.55s	remaining: 3.42s
312:	learn: 0.0026529	test: 0.0074101	best: 0.0074101 (312)	total: 1.55s	remaining: 3.41s
313:	learn: 0.0026441	test: 0.0073929	best: 0.0073929 (313)	total: 1.56s	remaining: 3.41s
314:	learn: 0.0026326	test: 0.0073500	best: 0.0073500 (314)	total: 1.56s	remaining: 3.4s
315:	learn: 0.0025916	test: 0.0073428	best: 0.0073428 (315)	total: 1.56s	remaining: 3.38s
316:	learn: 0.0025598	test: 0.0073595	best: 0.0073428 (315)	total: 1.57s	remaining: 3.37s
317:	learn: 0.0025294	test: 0.0074022	best: 0.0073428 (315)	total: 1.57s	remaining: 3.37s
318:	learn: 0.0025185	test: 0.0073606	best: 0.0073428 (315)	total: 1.57s	remaining: 3.36s
319:	learn: 0.0025105	test: 0.0073454	best: 0.0073428 (315)	total: 1.57s	remaining: 3.35s
320:	learn: 0.0024732	test: 0.0073256	best: 0.0073256 (320)	total: 1.58s	remaining: 3.34s
321:	learn: 0.0024632	test: 0.0072927	best: 0.0072927 (321)	total: 1.58s	remaining: 3.33s
322:	learn:

...

608:	learn: 0.0020582	test: 0.0067552	best: 0.0067552 (608)	total: 2.27s	remaining: 1.46s
609:	learn: 0.0020583	test: 0.0067552	best: 0.0067552 (609)	total: 2.27s	remaining: 1.45s
610:	learn: 0.0020582	test: 0.0067551	best: 0.0067551 (610)	total: 2.28s	remaining: 1.45s
611:	learn: 0.0020582	test: 0.0067551	best: 0.0067551 (611)	total: 2.28s	remaining: 1.44s
612:	learn: 0.0020582	test: 0.0067551	best: 0.0067551 (612)	total: 2.28s	remaining: 1.44s
613:	learn: 0.0020582	test: 0.0067551	best: 0.0067551 (613)	total: 2.28s	remaining: 1.44s
614:	learn: 0.0020582	test: 0.0067551	best: 0.0067551 (614)	total: 2.29s	remaining: 1.43s
615:	learn: 0.0020582	test: 0.0067550	best: 0.0067550 (615)	total: 2.29s	remaining: 1.43s
616:	learn: 0.0020582	test: 0.0067550	best: 0.0067550 (616)	total: 2.29s	remaining: 1.42s
617:	learn: 0.0020582	test: 0.0067550	best: 0.0067550 (617)	total: 2.29s	remaining: 1.42s
618:	learn: 0.0020582	test: 0.0067550	best: 0.0067550 (618)	total: 2.3s	remaining: 1.41s
619:	learn:

714:	learn: 0.0020353	test: 0.0067512	best: 0.0067512 (714)	total: 2.62s	remaining: 1.04s
715:	learn: 0.0020353	test: 0.0067512	best: 0.0067512 (715)	total: 2.63s	remaining: 1.04s
716:	learn: 0.0020352	test: 0.0067508	best: 0.0067508 (716)	total: 2.64s	remaining: 1.04s
717:	learn: 0.0020352	test: 0.0067508	best: 0.0067508 (717)	total: 2.64s	remaining: 1.04s
718:	learn: 0.0020352	test: 0.0067508	best: 0.0067508 (718)	total: 2.64s	remaining: 1.03s
719:	learn: 0.0020351	test: 0.0067507	best: 0.0067507 (719)	total: 2.65s	remaining: 1.03s
720:	learn: 0.0020352	test: 0.0067507	best: 0.0067507 (720)	total: 2.65s	remaining: 1.02s
721:	learn: 0.0020352	test: 0.0067507	best: 0.0067507 (721)	total: 2.65s	remaining: 1.02s
722:	learn: 0.0020351	test: 0.0067507	best: 0.0067507 (722)	total: 2.66s	remaining: 1.02s
723:	learn: 0.0020351	test: 0.0067507	best: 0.0067507 (723)	total: 2.66s	remaining: 1.01s
724:	learn: 0.0020351	test: 0.0067506	best: 0.0067506 (724)	total: 2.67s	remaining: 1.01s
725:	learn

876:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 2.99s	remaining: 419ms
877:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 2.99s	remaining: 416ms
878:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 2.99s	remaining: 412ms
879:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3s	remaining: 409ms
880:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3s	remaining: 405ms
881:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3s	remaining: 402ms
882:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3s	remaining: 398ms
883:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.01s	remaining: 395ms
884:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.01s	remaining: 391ms
885:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.01s	remaining: 388ms
886:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.02s	remaining: 384ms
887:	learn: 0.0020305	

968:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.17s	remaining: 101ms
969:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.17s	remaining: 98.1ms
970:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.17s	remaining: 94.8ms
971:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.18s	remaining: 91.5ms
972:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.18s	remaining: 88.2ms
973:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.18s	remaining: 84.9ms
974:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.19s	remaining: 81.7ms
975:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.19s	remaining: 78.4ms
976:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.19s	remaining: 75.1ms
977:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.19s	remaining: 71.8ms
978:	learn: 0.0020305	test: 0.0067474	best: 0.0067474 (798)	total: 3.19s	remaining: 68.5ms


0.9968544857768052